In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
# ifc_to_brick_minio.py — IFC → Brick TTL (avec BIM) + upload MinIO

import os, sys, argparse, json
from pathlib import Path
import ifcopenshell
import ifcopenshell.util.element as iutil
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS, XSD

# ========= MinIO =========
def s3_client(endpoint, access, secret, secure):
    import boto3
    from botocore.config import Config
    return boto3.client(
        "s3",
        endpoint_url=endpoint,
        aws_access_key_id=access,
        aws_secret_access_key=secret,
        use_ssl=bool(secure), verify=bool(secure),
        region_name="us-east-1", config=Config(signature_version="s3v4"),
    )

def ensure_bucket(s3, bucket):
    import botocore
    try: s3.head_bucket(Bucket=bucket)
    except botocore.exceptions.ClientError: s3.create_bucket(Bucket=bucket)

def s3_upload(s3, bucket, p:Path, key:str):
    s3.upload_file(str(p), bucket, key)

# ========= Namespaces =========
BRICK = Namespace("https://brickschema.org/schema/1.1/Brick#")
BF    = Namespace("https://brickschema.org/schema/BrickFrame#")
QUDT  = Namespace("http://qudt.org/schema/qudt/")
UNIT  = Namespace("http://qudt.org/vocab/unit/")
EX    = Namespace("http://example.org/training#")

SENSOR_MAP = [
    ("Temp_Sensor",     BRICK.Temperature_Sensor,        UNIT.DEG_C,    "sensors/{month}/zone_101_sensors.csv"),
    ("Humidity_Sensor", BRICK.Relative_Humidity_Sensor,  UNIT.PERCENT,  "sensors/{month}/zone_101_sensors.csv"),
    ("CO2_Sensor",      BRICK.CO2_Sensor,                UNIT.PPM,      "sensors/{month}/zone_101_sensors.csv"),
    ("Occupancy_Sensor",BRICK.Occupancy_Sensor,          UNIT.UNITLESS, "occupancy/{month}/occupancy.csv"),
]
EQUIP_MAP = [
    ("AC_Unit",       BRICK.Air_Conditioner, "HVAC_System", BRICK.HVAC_System),
    ("LED_Luminaire", BRICK.Luminaire,       "Lighting_System", BRICK.Lighting_System),
]

def add(g, s, p, o): g.add((s,p,o))
def label(g, s, txt): add(g, s, RDFS.label, Literal(txt))

def add_ex_dbl(g, subj, pred, val, unit_uri=None):
    if val is None: return
    add(g, subj, pred, Literal(float(val), datatype=XSD.double))
    if unit_uri is not None:
        add(g, subj, QUDT.unit, unit_uri)

def ifc_to_brick(ifc_path:Path, month:str, raw_prefix_minio:str) -> Graph:
    m = ifcopenshell.open(str(ifc_path))
    g = Graph()
    for pfx, ns in [("brick",BRICK),("bf",BF),("qudt",QUDT),("unit",UNIT),("ex",EX),("rdfs",RDFS)]:
        g.bind(pfx, ns)

    # Campus/Building/Room
    campus   = EX["Campus_ENSMR"];  add(g, campus, RDF.type, BRICK.Campus);   label(g, campus, "ENSMR Campus")
    building = EX["Building_ENSMR"]; add(g, building, RDF.type, BRICK.Building); label(g, building, "Teaching Building")
    add(g, campus, BRICK.hasPart, building)

    # Room from IFC
    room_name = "Room_101"; room_ifc = None
    for sp in m.by_type("IfcSpace"):
        room_ifc = sp
        if sp.Name: room_name = str(sp.Name).replace(" ", "_")
        break
    room = EX[room_name]; add(g, room, RDF.type, BRICK.Room); label(g, room, room_name.replace("_"," "))
    add(g, building, BRICK.hasPart, room)

    # === BIM: propriétés de la salle (PSet_TrainingRoom) ===
    if room_ifc is not None:
        psets = iutil.get_psets(room_ifc) or {}
        ps = psets.get("PSet_TrainingRoom") or psets.get("PSet_Trainingroom") or {}
        def take(*keys): 
            for k in keys:
                if k in ps: return ps[k]
            return None
        add_ex_dbl(g, room, EX.length_m,  take("Length_m","Length","Longueur"), UNIT.M)
        add_ex_dbl(g, room, EX.width_m,   take("Width_m","Width","Largeur"),    UNIT.M)
        add_ex_dbl(g, room, EX.height_m,  take("Height_m","Height","Hauteur"),  UNIT.M)
        add_ex_dbl(g, room, EX.area_m2,   take("Area_m2","Area","Surface"),     UNIT.M2)
        add_ex_dbl(g, room, EX.volume_m3, take("Volume_m3","Volume"),           UNIT.M3)
        maxocc = take("MaxOccupancy","OccupationMaximale")
        if maxocc is not None: add(g, room, EX.max_occupancy, Literal(int(maxocc), datatype=XSD.integer))
        usetype = take("UseType","TypeUsage")
        if usetype is not None: add(g, room, EX.use_type, Literal(str(usetype), datatype=XSD.string))
        add_ex_dbl(g, room, EX.target_occupancy_rate, take("TargetOccupancyRate","TauxOccupation"), UNIT.PERCENT)

    # Systèmes
    sys_nodes = {}
    for sys_name, sys_cls in [("HVAC_System", BRICK.HVAC_System),
                              ("Lighting_System", BRICK.Lighting_System),
                              ("Environmental_System", BRICK.Building_System)]:
        s = EX[sys_name]; add(g, s, RDF.type, sys_cls); sys_nodes[sys_name] = s

    # Équipements et BIM (PSet_HVAC, PSet_Lighting)
    equip_nodes = {}
    for ifcprod in m.by_type("IfcElement"):
        nm = (ifcprod.Name or "").lower()
        for key, brick_cls, sys_name, _sys_cls in EQUIP_MAP:
            if key.lower() in nm:
                e = EX[key + "_1"]
                add(g, e, RDF.type, brick_cls); label(g, e, key.replace("_"," ") + " 1")
                add(g, e, BRICK.hasLocation, room)
                add(g, sys_nodes[sys_name], BRICK.hasPart, e)
                equip_nodes[key] = e
                # BIM: lire PSet
                psets = iutil.get_psets(ifcprod) or {}
                if key.startswith("AC_Unit"):
                    ph = psets.get("PSet_HVAC", {})
                    add_ex_dbl(g, e, EX.thermal_power_kW,   ph.get("ThermalPower_kW"), UNIT.KiloW)
                    add_ex_dbl(g, e, EX.electrical_power_kW,ph.get("ElectricalPower_kW"), UNIT.KiloW)
                    mode = ph.get("Mode"); 
                    if mode: add(g, e, EX.mode, Literal(str(mode), datatype=XSD.string))
                    cdate = ph.get("CommissioningDate")
                    if cdate: add(g, e, EX.commissioning_date, Literal(str(cdate), datatype=XSD.date))
                if key.startswith("LED_Luminaire"):
                    pl = psets.get("PSet_Lighting", {})
                    add_ex_dbl(g, e, EX.rated_flux_lm,          pl.get("RatedFlux_lm"), UNIT.Lumen)
                    add_ex_dbl(g, e, EX.measured_illuminance_lux, pl.get("MeasuredIlluminance_lux"), UNIT.Lux)
                    add_ex_dbl(g, e, EX.electric_power_kW,      pl.get("ElectricPower_kW"), UNIT.KiloW)
                    typ = pl.get("Type")
                    if typ: add(g, e, EX.lighting_type, Literal(str(typ), datatype=XSD.string))

    # Murs + propriétés thermiques (PSet_Wall_Thermal) + orientation simple
    walls_ifc = [w for w in m.by_type("IfcWall")]
    orientations = ["North","East","South","West"]
    for i, w in enumerate(walls_ifc):
        wname = f"Wall_{i+1}"
        wx = EX[wname]
        add(g, wx, RDF.type, EX.Wall); label(g, wx, wname)
        add(g, building, BRICK.hasPart, wx)
        # Orientation approximative si géométrie absente
        add(g, wx, EX.orientation, Literal(orientations[i % 4], datatype=XSD.string))
        psets = iutil.get_psets(w) or {}
        pw = psets.get("PSet_Wall_Thermal", {})
        thick = pw.get("Thickness_m") or pw.get("Thickness") or 0.3
        r_val = pw.get("R_Value_m2K_W") or pw.get("R_Value") or pw.get("ResistanceThermique")
        cond  = pw.get("Conductivity_W_mK") or pw.get("Conductivite")
        add_ex_dbl(g, wx, EX.thickness_m, thick, UNIT.M)
        if r_val is not None:
            add_ex_dbl(g, wx, EX.r_value_m2K_W, r_val, UNIT.M2__K__W)  # m²·K/W
            try:
                u = 1.0/float(r_val) if float(r_val) > 0 else None
            except Exception:
                u = None
            if u is not None:
                add_ex_dbl(g, wx, EX.u_value_W_m2K, u, UNIT.W__M2__K)
        if cond is not None:
            add_ex_dbl(g, wx, EX.conductivity_W_mK, cond, UNIT.W__M__K)
        vt = pw.get("VisibleTransmittance")
        if vt is not None:
            add_ex_dbl(g, wx, EX.visible_transmittance, vt, UNIT.UNITLESS)

    # Capteurs (timeseries → RAW URIs)
    for ifcprod in m.by_type("IfcSensor"):
        nm = (ifcprod.Name or "")
        for key, brick_cls, unit_uri, relpath_tpl in SENSOR_MAP:
            if key in nm:
                sx = EX[key + "_1"]
                add(g, sx, RDF.type, brick_cls); label(g, sx, key.replace("_"," ") + " 1")
                add(g, sx, BRICK.hasLocation, room)
                if "Temp_" in key or "Humidity_" in key or "CO2_" in key:
                    if "AC_Unit" in equip_nodes: add(g, sx, BRICK.isPointOf, equip_nodes["AC_Unit"])
                if "Occupancy_" in key:
                    if "LED_Luminaire" in equip_nodes: add(g, sx, BRICK.isPointOf, equip_nodes["LED_Luminaire"])
                add(g, sx, QUDT.unit, unit_uri)
                ts_rel = relpath_tpl.format(month=month)
                add(g, sx, BF.hasTimeseriesId, Literal(f"{raw_prefix_minio}/{ts_rel}", datatype=XSD.string))
                add(g, EX["Environmental_System"], BRICK.hasPoint, sx)

    return g

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--month", type=str, default="2025-03")
    ap.add_argument("--ifc_base", type=str, default="~/DTE/jne_project/raw/ifc")
    ap.add_argument("--ref_base", type=str, default="~/DTE/jne_project/refined")
    ap.add_argument("--raw_minio_prefix", type=str, default="minio://raw")
    # MinIO refined
    ap.add_argument("--endpoint", type=str, default=os.environ.get("MINIO_ENDPOINT","http://192.168.0.173:9000"))
    ap.add_argument("--access",   type=str, default=os.environ.get("MINIO_ROOT_USER","minioadmin"))
    ap.add_argument("--secret",   type=str, default=os.environ.get("MINIO_ROOT_PASSWORD","minioadmin"))
    ap.add_argument("--bucket",   type=str, default="refined")
    ap.add_argument("--prefix",   type=str, default="jne_project/refined")
    ap.add_argument("--secure",   action="store_true")
    ap.add_argument("--no-upload", action="store_true")
    args,_ = ap.parse_known_args()

    ifc_path = Path(args.ifc_base).expanduser().resolve()/args.month/"training_room.ifc"
    out_dir  = Path(args.ref_base).expanduser().resolve()/ "brick"/args.month
    meta_dir = Path(args.ref_base).expanduser().resolve()/ "meta"/args.month
    out_dir.mkdir(parents=True, exist_ok=True); meta_dir.mkdir(parents=True, exist_ok=True)

    g = ifc_to_brick(ifc_path, args.month, args.raw_minio_prefix)
    ttl_path = out_dir/"training_room.ttl"
    g.serialize(destination=str(ttl_path), format="turtle")

    manifest = {
        "version":"1.1",
        "month": args.month,
        "input_ifc": str(ifc_path),
        "output_ttl": str(ttl_path),
        "namespaces":{"brick":str(BRICK),"bf":str(BF),"qudt":str(QUDT),"unit":str(UNIT),"ex":str(EX)},
        "timeseries_prefix": args.raw_minio_prefix,
        "bim_fields_room":[
            "ex:length_m","ex:width_m","ex:height_m","ex:area_m2","ex:volume_m3",
            "ex:max_occupancy","ex:use_type","ex:target_occupancy_rate"
        ],
        "bim_fields_walls":[
            "ex:orientation","ex:thickness_m","ex:r_value_m2K_W","ex:u_value_W_m2K",
            "ex:conductivity_W_mK","ex:visible_transmittance"
        ],
        "bim_fields_equipment":[
            "ex:thermal_power_kW","ex:electrical_power_kW","ex:mode","ex:commissioning_date",
            "ex:rated_flux_lm","ex:measured_illuminance_lux","ex:electric_power_kW","ex:lighting_type"
        ]
    }
    man_path = meta_dir/"brick_manifest.json"
    man_path.write_text(json.dumps(manifest, indent=2), encoding="utf-8")

    if not args.no_upload:
        try:
            s3 = s3_client(args.endpoint, args.access, args.secret, args.secure)
            ensure_bucket(s3, args.bucket)
            root = args.prefix.strip("/")
            s3_upload(s3, args.bucket, ttl_path, f"{root}/brick/{args.month}/training_room.ttl")
            s3_upload(s3, args.bucket, man_path, f"{root}/meta/{args.month}/brick_manifest.json")
            print("minio:", f"s3://{args.bucket}/{root}/brick/{args.month}/training_room.ttl")
        except Exception as e:
            print("ERREUR MinIO:", e, file=sys.stderr); sys.exit(3)

    print("local:", ttl_path, "|", man_path)

if __name__ == "__main__":
    main()


minio: s3://refined/jne_project/refined/brick/2025-03/training_room.ttl
local: /home/amina/DTE/jne_project/refined/brick/2025-03/training_room.ttl | /home/amina/DTE/jne_project/refined/meta/2025-03/brick_manifest.json
